In [1]:
!pip install langchain langchain-huggingface huggingface_hub
!pip install sentence-transformers chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 7.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 9.4 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: nvidia-

In [2]:
# pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"

device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

2025-05-17 07:05:38.854480: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747465539.045248      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747465539.105195      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

## Setup

Let's start by importing the necessary libraries and setting up our environment.

## Basic Chain of Thought Prompting

Let's start with a simple example to demonstrate the difference between a standard prompt and a Chain of Thought prompt.

In [3]:
%pip install transformers langchain langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22
Note: you may need to restart the kernel to use updated packages.


In [135]:
from datasets import load_dataset

ds = load_dataset("qiaojin/PubMedQA", "pqa_artificial")



In [137]:
ds = ds['train'].select_columns(["question", "context", "long_answer", "final_decision"])
ds = ds.select(range(30))
print(ds)

small_dataset = ds.to_list()

print(f"Loaded {len(small_dataset)} processed samples.")

Dataset({
    features: ['question', 'context', 'long_answer', 'final_decision'],
    num_rows: 30
})
Loaded 30 processed samples.


In [52]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = ["\n\n".join(doc["context"]['contexts']) for doc in small_dataset]

embeddings = embedding_model.embed_documents(texts)

vector_store = FAISS.from_texts(texts, embedding_model)

print("Embeddings stored in FAISS.")

5000
Embeddings stored in FAISS.


In [85]:
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.prompts import PromptTemplate
import re

def retrieve_context(state):
    print("retriveing")
    """Retrieve relevant context using FAISS."""
    retrieved_docs = vector_store.similarity_search(state["question"], k=3)
    state["context"] = "\n".join([doc.page_content for doc in retrieved_docs])
    
    return state

retrieval = RunnableLambda(retrieve_context)



def generate_response(prompt):
    print("generating")
    messages = [{"role": "user", "content": f"{prompt}"}]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=200, temperature=0.6, top_p=0.92, do_sample=True)
    
    return  tokenizer.decode(outputs[0], skip_special_tokens=True).replace("'", "")

generation = RunnableLambda(generate_response)


In [90]:
standard_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=""""
Context: {context}
----
Question: {question}
    """
)

# Chain of Thought Prompt
cot_prompt = PromptTemplate(
input_variables=["question","context"],
template="""
Guidelines
----------
1- You are a medical AI assistant.
2- Provide accurate step by step answer.
3- Analyze and reason the question.
4- Answer the question based on the provided context
5- Answer all mentioned points in the question
6- Refer to the context provided when explaining and reasoning answer
.
Context: 
-------
{context}

Answer the following Question based on provided guidelines

{question}
    """
)

In [96]:
cot_prompt_based_pipeline = retrieval | cot_prompt | generation

zero_shot_prompt_based_pipeline = retrieval | standard_prompt | generation


# Example question
question_data = {"question": "Are group 2 innate lymphoid cells ( ILC2s ) increased in chronic rhinosinusitis with nasal polyps or eosinophilia?"}

cot_response = cot_prompt_based_pipeline.invoke(question_data)

# # Execute pipeline
zero_shot_response = zero_shot_prompt_based_pipeline.invoke(question_data)

print("\n🔹 Final RAG Response:", zero_shot_response.replace("\\n", "\n"))
# print("\n🔹 Final RAG Response:", cot_response.replace("\\n", "\n"))

retriveing
generating
retriveing
generating

🔹 Final RAG Response: user
text="
Context: Chronic rhinosinusitis (CRS) is a heterogeneous disease with an uncertain pathogenesis. Group 2 innate lymphoid cells (ILC2s) represent a recently discovered cell population which has been implicated in driving Th2 inflammation in CRS; however, their relationship with clinical disease characteristics has yet to be investigated.

The aim of this study was to identify ILC2s in sinus mucosa in patients with CRS and controls and compare ILC2s across characteristics of disease.

A cross-sectional study of patients with CRS undergoing endoscopic sinus surgery was conducted. Sinus mucosal biopsies were obtained during surgery and control tissue from patients undergoing pituitary tumour resection through transphenoidal approach. ILC2s were identified as CD45(+) Lin(-) CD127(+) CD4(-) CD8(-) CRTH2(CD294)(+) CD161(+) cells in single cell suspensions through flow cytometry. ILC2 frequencies, measured as a perc

In [150]:
!pip install evaluate rouge_score bleu meteor rouge_scorer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement rouge_scorer (from versions: none)
ERROR: No matching distribution found for rouge_scorer


In [ ]:
import re
import string
from collections import Counter
from tqdm import tqdm
from dotenv import load_dotenv
import os
import pandas as pd
from collections import Counter
import re
import string
import unicodedata
import numpy as np 
import evaluate
from rouge_score import rouge_scorer

# Normalize text for EM/F1
def normalize_text(text):
    def remove_articles(s):
        return re.sub(r'\b(a|an|the)\b', ' ', s)

    def white_space_fix(s):
        return ' '.join(s.split())

    def remove_punc(s):
        return ''.join(ch for ch in s if ch not in string.punctuation)

    def lower(s):
        return s.lower()

    def unicode_to_ascii(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    return white_space_fix(remove_articles(remove_punc(lower(unicode_to_ascii(text)))))
    
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

def get_metrics(df):

     

    # BLEU
    bleu_result = bleu.compute(predictions=predictions, references=[[a] for a in answers])
    # ROUGE
    rouge_result = rouge.compute(predictions=predictions, references=answers)

    return em_scores, f1_scores, bleu_result, rouge_result

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    return 2 * (precision * recall) / (precision + recall)

def evaluate(dataset, prompting_type):
    f1_scores = []
    for item in small_dataset:
        print("1..")
        question = {"question":item["question"]}
        prediction = cot_prompt_based_pipeline.invoke(question).replace("\\n", "")
        ground_truth = item['long_answer']
        
        pred_tokens = normalize_text(prediction).split()
        gt_tokens = normalize_text(ground_truth).split()
        
        common = Counter(pred_tokens) & Counter(gt_tokens)
        num_same = sum(common.values())
        if num_same == 0:
            return 0.0
        precision = num_same / len(pred_tokens)
        recall = num_same / len(gt_tokens)
        f1_scores.append(2 * precision * recall / (precision + recall))
    return  np.mean(f1_scores)
    
def eval_rouge(dataset, prompting_type):
 # Initialize ROUGE scorer
    for item in small_dataset:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        question = {"question":item["question"]}
        prediction = cot_prompt_based_pipeline.invoke(question).replace("\\n", "")
        ground_truth = item['long_answer']
        # Compute scores
        scores = scorer.score(ground_truth, prediction)
        
        # Print results
        print("ROUGE-1:", scores['rouge1'])
        print("ROUGE-2:", scores['rouge2'])
        print("ROUGE-L:", scores['rougeL'])
# F1 Score
# def f1_score(prediction, ground_truth):
#     pred_tokens = normalize_text(prediction).split()
#     gt_tokens = normalize_text(ground_truth).split()
#     common = Counter(pred_tokens) & Counter(gt_tokens)
#     num_same = sum(common.values())
#     if num_same == 0:
#         return 0.0
#     precision = num_same / len(pred_tokens)
#     recall = num_same / len(gt_tokens)
#     return 2 * precision * recall / (precision + recall)

    
print(eval_rouge(small_dataset, "cot"))

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


retriveing
generating
ROUGE-1: Score(precision=0.05239179954441914, recall=0.8214285714285714, fmeasure=0.09850107066381156)
ROUGE-2: Score(precision=0.021664766248574687, recall=0.34545454545454546, fmeasure=0.04077253218884121)
ROUGE-L: Score(precision=0.03302961275626424, recall=0.5178571428571429, fmeasure=0.06209850107066381)
retriveing
generating
ROUGE-1: Score(precision=0.02386634844868735, recall=0.9090909090909091, fmeasure=0.04651162790697674)
ROUGE-2: Score(precision=0.014336917562724014, recall=0.5714285714285714, fmeasure=0.027972027972027972)
ROUGE-L: Score(precision=0.02028639618138425, recall=0.7727272727272727, fmeasure=0.03953488372093023)
retriveing
generating
ROUGE-1: Score(precision=0.08347826086956522, recall=0.5, fmeasure=0.14307004470938897)
ROUGE-2: Score(precision=0.013937282229965157, recall=0.08421052631578947, fmeasure=0.02391629297458894)
ROUGE-L: Score(precision=0.050434782608695654, recall=0.3020833333333333, fmeasure=0.08643815201192251)
retriveing
gene